# 🐶 Introduction

In this competition, the task is to predict the "Pawpularity Score" given an input image. The score range from 0-100 and thus can be formulated as classification problem or regression problem. 

# 🐕 About this kernel

This kernel fine-tunes an EfficientNetv2 model using TensorFlow and uses Weights and Biases (W&B) for efficient experiment tracking. Meta-data is also used for training. 

In the past, I have created multiple training kernels that use Weights and Biases but this kernel introduces some useful W&B best-practices (framework agnostic) that you can easily incorporate in your own pipeline with slight to no modification. 

In particular, this kernel introduces:

1. Use of hash value (random string) for efficient grouping of experiments in the W&B UI. ✔️ 
2. Fine-tuning of EfficientNetv2 hosted on TensorFlow Hub. 
3. Use of Albumentations with TensorFlow for data augmentation. 
4. Track metrics using W&B Experiment Tracking. ✔️ 
4. Keep track of the model and `oof.csv` file using W&B Artifacts.✔️ 
5. Save the trained model as Kaggle dataset to be used for inference.
6. Visualize model perforance using W&B Tables. ✔️ 
7. Manually log LB Score to Weights and Biases. ✔️ 

The bullet points marked with ✔️ are specific to Weights and Biases and can be incorporated in your own workflow. 

# 🦮 Imports and Setups

* Install the latest version of W&B. 
* Generate random string as hash value.
* Import important libraries.
* Loging to W&B using your W&B API access token.

In [ ]:
!pip install -q --upgrade wandb

### ❓ Why is this useful?

🙋 In Kaggle we usually train model using K-fold strategy. If you are using W&B for experiment tracking you can see aggregated training metrics for each fold if you pass in `group=some-name` argument to `wandb.init()`. In the context of Kaggle, where we keep trying new experiments, tweak hyperparameters, change model definition, etc. regularly, deciding on this `some-name` can be tricky and prone to human error (usual error is to forget renaming for a new experiment). 

By using a randomly generated hash-value you can group one experiment under one group-name and name the model as well as any generated file with that name. This is an unique identifier about your experiment. 

⚠️ Run this cell at the start of the notebook.

In [ ]:
import string
import random
def id_generator(size=12, chars=string.ascii_lowercase + string.digits):
    return ''.join(random.SystemRandom().choice(chars) for _ in range(size))

HASH_NAME = id_generator(size=12)
print(HASH_NAME)

Each experiments are grouped together using the hash-value. 

![img](https://i.imgur.com/a7atecz.png)

In [ ]:
import os
# Close TF debug logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import gc
import sys
import time
import json
import wandb
print("W&B version: ", wandb.__version__)

import signal
import numpy as np
import pandas as pd
from tqdm import tqdm
from functools import partial
import matplotlib.pyplot as plt
from wandb.keras import WandbCallback

# For deep learning
import tensorflow as tf
print("TF version: ", tf.__version__)
import tensorflow_hub as hub
print('Hub version:', hub.__version__)

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import regularizers
import tensorflow_addons as tfa
import tensorflow_probability as tfp
tfd = tfp.distributions

# Sklearn for spliting the dataset
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

# Imports for augmentations. 
from albumentations import Compose, RandomResizedCrop, Cutout, Rotate, HorizontalFlip, VerticalFlip, RandomBrightnessContrast, ShiftScaleRotate, CenterCrop, Resize, Normalize

### ❓ Why is this useful?

🙋 In case a cell with initialized W&B run is KeyboardInterrupted, this function will close the W&B run. If you use a Python script to train your model, W&B will close automatically if you interrupt the training process. 

In [ ]:
def wandbKeyboardInterruptHandler(signal, frame):
    print("KeyboardInterrupt (ID: {}) has been caught. Cleaning up...".format(signal))
    wandb.finish()
    sys.exit()

#### Utils for EfficientNetV2

In [ ]:
# For efficientnet class of models
def get_hub_url_and_isize(model_name, ckpt_type, hub_type):
  if ckpt_type == '1k':
    ckpt_type = ''  # json doesn't support empty string
  else:
    ckpt_type = '-' + ckpt_type  # add '-' as prefix
  
  hub_url_map = {
    'efficientnetv2-b0': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0/{hub_type}',
    'efficientnetv2-b1': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1/{hub_type}',
    'efficientnetv2-b2': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2/{hub_type}',
    'efficientnetv2-b3': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3/{hub_type}',
    'efficientnetv2-s':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s/{hub_type}',
    'efficientnetv2-m':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m/{hub_type}',
    'efficientnetv2-l':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l/{hub_type}',

    'efficientnetv2-b0-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0-21k/{hub_type}',
    'efficientnetv2-b1-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1-21k/{hub_type}',
    'efficientnetv2-b2-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2-21k/{hub_type}',
    'efficientnetv2-b3-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3-21k/{hub_type}',
    'efficientnetv2-s-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s-21k/{hub_type}',
    'efficientnetv2-m-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m-21k/{hub_type}',
    'efficientnetv2-l-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l-21k/{hub_type}',
    'efficientnetv2-xl-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-xl-21k/{hub_type}',

    'efficientnetv2-b0-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0-21k-ft1k/{hub_type}',
    'efficientnetv2-b1-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1-21k-ft1k/{hub_type}',
    'efficientnetv2-b2-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2-21k-ft1k/{hub_type}',
    'efficientnetv2-b3-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3-21k-ft1k/{hub_type}',
    'efficientnetv2-s-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s-21k-ft1k/{hub_type}',
    'efficientnetv2-m-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m-21k-ft1k/{hub_type}',
    'efficientnetv2-l-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l-21k-ft1k/{hub_type}',
    'efficientnetv2-xl-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-xl-21k-ft1k/{hub_type}',
      
    # efficientnetv1
    'efficientnet_b0': f'https://tfhub.dev/tensorflow/efficientnet/b0/{hub_type}/1',
    'efficientnet_b1': f'https://tfhub.dev/tensorflow/efficientnet/b1/{hub_type}/1',
    'efficientnet_b2': f'https://tfhub.dev/tensorflow/efficientnet/b2/{hub_type}/1',
    'efficientnet_b3': f'https://tfhub.dev/tensorflow/efficientnet/b3/{hub_type}/1',
    'efficientnet_b4': f'https://tfhub.dev/tensorflow/efficientnet/b4/{hub_type}/1',
    'efficientnet_b5': f'https://tfhub.dev/tensorflow/efficientnet/b5/{hub_type}/1',
    'efficientnet_b6': f'https://tfhub.dev/tensorflow/efficientnet/b6/{hub_type}/1',
    'efficientnet_b7': f'https://tfhub.dev/tensorflow/efficientnet/b7/{hub_type}/1',
  }
  
  image_size_map = {
    'efficientnetv2-b0': 224,
    'efficientnetv2-b1': 240,
    'efficientnetv2-b2': 260,
    'efficientnetv2-b3': 300,
    'efficientnetv2-s':  384,
    'efficientnetv2-m':  480,
    'efficientnetv2-l':  480,
    'efficientnetv2-xl':  512,
  
    'efficientnet_b0': 224,
    'efficientnet_b1': 240,
    'efficientnet_b2': 260,
    'efficientnet_b3': 300,
    'efficientnet_b4': 380,
    'efficientnet_b5': 456,
    'efficientnet_b6': 528,
    'efficientnet_b7': 600,
  }
  
  hub_url = hub_url_map.get(model_name + ckpt_type)
  image_size = image_size_map.get(model_name, 224)
  return hub_url, image_size

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

#### Login to Weights and Biases

If you are using Kaggle kernel to train your model and commit the kernel, it's best to use Kaggle User Secrets. Put in your W&B Access token with the label `wandb_api`. 

If you Quick Save your kernel you can simply use `wandb.login()`.

If you are using a local system (where the runtime is not volatile) you just need to call `wandb.login()` once. 

In [ ]:
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb_api")
    wandb.login(key=secret_value_0)
    anony=None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

#  🐕‍🦺Hyperparameters

Save your hyperparameters as a dictionary so that you can log the same to W&B. 

In [ ]:
TRAIN_PATH = '../input/petfinder-pawpularity-score/train/'
AUTOTUNE = tf.data.experimental.AUTOTUNE

CONFIG = dict (
    seed = 42,
    loss = 'mse', # binary_crossentropy
    num_labels = 1,
    num_folds = 5,
    train_val_split = 0.2,
    img_width = 224,
    img_height = 224,
    batch_size = 16,
    epochs = 1, # FOR FULL TRAINING INCREASE EPOCHS TO ATLEAST 20.
    learning_rate = 1e-5,
    hash_name = HASH_NAME
)

# Model configs
CONFIG['model_type'] = 'efficientnetv2-b0'
CONFIG['ckpt_type'] = '1k'   # '21k', '21k-ft1k', '1k'
CONFIG['hub_type'] = 'feature-vector' # 
hub_url, image_size = get_hub_url_and_isize(CONFIG['model_type'], CONFIG['ckpt_type'], CONFIG['hub_type'])
print(hub_url)

CONFIG['group'] = f'{HASH_NAME}-Regression'
CONFIG['model_name'] = f'{HASH_NAME}-model'
CONFIG['img_width'] = image_size
CONFIG['img_height'] = image_size
CONFIG['do_fine_tuning'] = True

# Set the random seeds
def seed_everything(SEED):
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
    np.random.seed(SEED)
    tf.random.set_seed(SEED)
    
seed_everything(CONFIG['seed'])

# 🐱 Build Input Pipeline

In [ ]:
df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
print(f'Number of train images: {len(df)}')

# Add path
df['img_path'] = df['Id'].apply(lambda x: f'{TRAIN_PATH}{x}.jpg')

# Min-max scaling (For regression)
column = 'Pawpularity'
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())   

df.head()

In [ ]:
FEATURES = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']

In [ ]:
def create_folds(data, num_splits):
    data["kfold"] = -1
    num_bins = int(np.floor(1 + np.log2(len(data))))

    data.loc[:, "bins"] = pd.cut(data["Pawpularity"], bins=num_bins, labels=False)

    kf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    
    data = data.drop("bins", axis=1)

    return data

df = create_folds(df, num_splits=CONFIG['num_folds'])
df.to_csv('five_fold_classification.csv', index=False)

# 🐈 Dataloader

In [ ]:
# Apply training augmentation
train_transforms = Compose([
            Resize(CONFIG['img_height'], CONFIG['img_width'], p=1),
            Rotate(limit=20),
            Cutout(num_holes=8, max_h_size=30, max_w_size=30, p=1.0),
            HorizontalFlip(p=0.7),
            VerticalFlip(p=0.4),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0,
            ),
        ])

# Apply validation augmentation
valid_transforms = Compose([
            Resize(CONFIG['img_height'], CONFIG['img_width'], p=1),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0,
            ),
        ])

def aug_train_fn(image):
    data = {"image":image}
    aug_data = train_transforms(**data)
    aug_img = aug_data["image"]

    return aug_img.astype(np.float32) 

def aug_valid_fn(image):
    data = {"image":image}
    aug_data = valid_transforms(**data)
    aug_img = aug_data["image"]

    return aug_img.astype(np.float32) 

def train_augmentations(inputs, label):
    image = inputs['image']
    aug_img = tf.numpy_function(func=aug_train_fn, inp=[image], Tout=tf.float32)
    aug_img.set_shape((CONFIG['img_height'], CONFIG['img_width'], 3))

    return {'image': aug_img, 'features': inputs['features']}, label

def valid_augmentations(inputs, label):
    image = inputs['image']
    aug_img = tf.numpy_function(func=aug_valid_fn, inp=[image], Tout=tf.float32)
    aug_img.set_shape((CONFIG['img_height'], CONFIG['img_width'], 3))

    return {'image': aug_img, 'features': inputs['features']}, label

In [ ]:
@tf.function
def load_resize(df_dict):
    # Load image
    img = tf.io.read_file(df_dict['img_path'])
    img = tf.image.decode_jpeg(img, channels=3)
    
    # Parse features
    features = {key: df_dict[key] for key in FEATURES}
    features = features.values()
    
    # Parse label
    label = df_dict['Pawpularity']
    label = tf.cast(label, tf.float32)
    
    return {'image': img, 'features': features}, label

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

def get_dataloaders(train_df, valid_df):
    # Train Loader
    trainloader = tf.data.Dataset.from_tensor_slices(dict(train_df))
    # Valid Loader
    validloader = tf.data.Dataset.from_tensor_slices(dict(valid_df))

    trainloader = (
        trainloader
        .shuffle(1024)
        .map(load_resize, num_parallel_calls=AUTOTUNE)
        .map(train_augmentations, num_parallel_calls=AUTOTUNE)
        .batch(CONFIG['batch_size'])
        .prefetch(AUTOTUNE)
    )

    validloader = (
        validloader
        .map(load_resize, num_parallel_calls=AUTOTUNE)
        .map(valid_augmentations, num_parallel_calls=AUTOTUNE)
        .batch(CONFIG['batch_size'])
        .prefetch(AUTOTUNE)
    )
    
    return trainloader, validloader

In [ ]:
def show_batch(image_batch):
  plt.figure(figsize=(20,20))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.axis('off')

#sanity check
# Prepare dataloaders
train_df = df.loc[df.kfold != 0].reset_index(drop=True)
valid_df = df.loc[df.kfold == 0].reset_index(drop=True)

trainloader, validloader = get_dataloaders(train_df, valid_df)
inputs, labels = next(iter(validloader))

# show_batch(imgs)

# 🐕 Model

In [ ]:
def get_model():
    # Setup backbone model
    base_model = hub.KerasLayer(hub_url, trainable=CONFIG['do_fine_tuning'])

    # Inputs
    image_inputs = layers.Input((CONFIG['img_height'], CONFIG['img_width'], 3), name='image')
    feature_inputs = layers.Input((len(FEATURES)), name='features')
    
    # Get image features
    img_features = base_model(image_inputs)
    img_features = layers.Dense(64, activation='selu', 
                               kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                               bias_regularizer=regularizers.l2(1e-4), 
                               activity_regularizer=regularizers.l2(1e-5))(img_features)
    
    # Get metadata features
    features = layers.Dense(64, activation='selu',
                            kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                            bias_regularizer=regularizers.l2(1e-4),
                            activity_regularizer=regularizers.l2(1e-5))(feature_inputs)
    
    # Concat features
    concat_features = layers.concatenate([img_features, features])
    
    x = layers.Dropout(0.5)(concat_features)
    x = layers.Dense(64, activation='selu', 
                     kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                     bias_regularizer=regularizers.l2(1e-4),
                     activity_regularizer=regularizers.l2(1e-5))(x)
    
    outputs = layers.Dense(1, activation='sigmoid', 
                           kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                           bias_regularizer=regularizers.l2(1e-4),
                           activity_regularizer=regularizers.l2(1e-5))(x)
    
    return models.Model([image_inputs, feature_inputs], outputs)

tf.keras.backend.clear_session()
model = get_model()
model.summary()

# 🐶 Callbacks

In [ ]:
# Callbacks
earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=2, verbose=0, mode='min',
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=CONFIG['learning_rate'])

# 🐕‍🦺 Train

⚠️ For demonstration purposes I am training the model for 1 epochs. 

In [ ]:
def get_predictions(model, validloader, valid_df):
    y_pred = []
    for image_batch, label_batch in validloader:
        preds = model.predict(image_batch)
        y_pred.extend(preds)
        
    valid_df['preds'] = y_pred
    
    return valid_df 

In [ ]:
oof_df = pd.DataFrame()

In [ ]:
# In case we interrupt the kernel
signal.signal(signal.SIGINT, wandbKeyboardInterruptHandler)

for fold in range(CONFIG['num_folds']):
    print('FOLD: ', fold)
    # Prepare train and valid df
    train_df = df.loc[df.kfold != fold].reset_index(drop=True)
    valid_df = df.loc[df.kfold == fold].reset_index(drop=True)

    # Prepare dataloaders
    trainloader, validloader = get_dataloaders(train_df, valid_df)

    # Initialize model
    tf.keras.backend.clear_session()
    model = get_model()

    # Compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate=CONFIG['learning_rate'])
    model.compile(optimizer, 
                  loss='binary_crossentropy', 
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])

    # Update CONFIG dict with the name of the model.
    print('Training configuration: ', CONFIG)

    # Initialize W&B run
    run = wandb.init(project='petfinder-train', 
                     config=CONFIG,
                     group=CONFIG['group'], 
                     job_type='train',
                     tags=['effnetv2', f'{HASH_NAME}'],
                     name=f'{HASH_NAME}-fold-{fold}')

    # Train
    _ = model.fit(trainloader, 
                  epochs=CONFIG['epochs'],
                  validation_data=validloader,
                  callbacks=[WandbCallback(),
                             earlystopper,
                             reduce_lr])


    # Evaluate
    loss, rsme = model.evaluate(validloader)
    wandb.log({'Val RSME': rsme})

    # Save model
    model_name = CONFIG['model_name']
    MODEL_PATH = f'models/{model_name}'
    os.makedirs(MODEL_PATH, exist_ok=True)
    count_models = len(os.listdir(MODEL_PATH))

    model.save(f'{MODEL_PATH}/{model_name}_{count_models}')

    # Get Prediction on validation set
    _oof_df = get_predictions(model, validloader, valid_df)
    oof_df = pd.concat([oof_df, _oof_df])

    # Close W&B run
    run.finish()

    del model, trainloader, validloader, _oof_df
    _ = gc.collect()

# Save oof as csv file
oof_df.to_csv('oof_preds.csv', index=False)

# 🐈 Compute CV Score and Save Model and OOF File as W&B Artifacts

In [ ]:
# Initialize W&B run
run = wandb.init(project='petfinder-train', 
                 config=CONFIG,
                 group=CONFIG['group'], 
                 job_type='eval',
                 name=f'{HASH_NAME}-eval')

# Compute CV score
oof_df_copy = oof_df.copy()
def correct_preds(row):
    return row.preds[0]

oof_df_copy['preds'] = oof_df_copy.apply(lambda row: correct_preds(row), axis=1)

metric = tf.keras.metrics.RootMeanSquaredError()
metric.update_state(oof_df_copy.Pawpularity.values, oof_df_copy.preds.values)
print(f'CV Score: {metric.result().numpy()}')

wandb.log({"CV Score": metric.result().numpy()})

# Log oof.csv as artifacts
model_artifacts = wandb.Artifact(f'{HASH_NAME}', type='model', metadata={'hash_name': HASH_NAME})
# Add oof_preds.csv
model_artifacts.add_file('oof_preds.csv')
# Add trained models
model_artifacts.add_dir('models/')
wandb.log_artifact(model_artifacts)

wandb.finish()

This way you can track which experiment trained which model and the corresponding `oof_preds.csv` file. 

![img](https://i.imgur.com/gOz1LNj.png)

# 🐱 Save the model as Kaggle dataset



In [ ]:
# Copy Kaggle API token to ~/.kaggle
! mkdir -p /root/.kaggle/
! cp ../input/apitoken/kaggle.json /root/.kaggle/kaggle.json

# Initialize dataset creation
! kaggle datasets init -p models/

!ls models/

%cat models/dataset-metadata.json

In [ ]:
import json
  
# Opening JSON file
with open('models/dataset-metadata.json', 'r') as openfile:
    # Reading from json file
    json_object = json.load(openfile)

    json_object['title'] = f'Petfinder {HASH_NAME}'
    json_object['id'] = f'petfinder-{HASH_NAME}'

with open("models/dataset-metadata.json", "w") as outfile:
    json.dump(json_object, outfile, indent=2)

In [ ]:
!kaggle datasets create -p models/ --dir-mode zip
# ! kaggle datasets version -p /kaggle/tmp/hpa_512x512_dataset -m "add rgb images"  --dir-mode tar

In [ ]:
DESP = "efficientnetv2-b0_trained_with_metadata"
!kaggle datasets version -p models -m {DESP} --dir-mode zip

# 🐕 Analysis of Model Performance

In [ ]:
import seaborn as sns
sns.set(style='dark')

plt.figure(figsize=(10*2,6*2))

# Prediction Plot
plt.subplot(2, 2, 1)
sns.kdeplot(x=oof_df_copy.Pawpularity.values, color='b',shade=True);
sns.kdeplot(x=oof_df_copy.preds.values, color='r',shade=True);
plt.grid('ON')
plt.xlabel(oof_df_copy.Pawpularity.values);plt.ylabel('freq');plt.title('KDE')
plt.legend(['train', 'oof'])

plt.tight_layout()
plt.show()

## Update LB Score

Use the trained model to run inference using a different kernel. Once you get the LB score, use the cell code below to log the LB score to the <HASH_NAME>-eval W&B run name. 

![img](https://i.imgur.com/c7CAlBH.gif)

In [ ]:
import wandb
api = wandb.Api()

run = api.run("ayut/petfinder-train/3sopqxw1")
run.summary["LB Score"] = 0.22954 # This is just representative number.
run.summary.update()